# NCAA Tweet Exploration
#### Katherine Taylor
The focus of this project is to generate fake tweets using team account data from the 2019 NCAA tournament first and second round. Two models will be built: one to produce the tweets, and the other to predict the number of likes, retweets, and replies the tweet would have received. The end goal is to create a joke NCAA basketball team account with several tweets. The below notebook explores the existing data.

### 1. Imports

In [20]:
from bokeh.layouts import gridplot
from bokeh.plotting import figure, output_notebook, show
import pandas as pd
import numpy as np
output_notebook()

Loading BokehJS ...

In [6]:
# import data
basketball_tweets = pd.read_csv("data/cleaned_team_data.csv")

In [8]:
basketball_tweets.head()

,Unnamed: 0,ID,AuthorUserName,AuthorDisplayName,Created,Message,ContentType,ContextUrl,AuthorFollowerCount,LikeCount,TwitterAuthorListedCount,TWAuthorIsVerified,TWRetweetCount,TWReplyCount,TWAuthorCreated,Tags,ZPoints,Sentiment,No.Tags,Engagement.Score
0,1,1.109240e+18,LibertyMBB,Liberty Menâ€™s Basketball,2019-03-22 23:59:00,WELCOME TO THE DANCE MR. CABBIL The Senior has...,Video,https://twitter.com/LibertyMBB/statuses/110924...,8377,14,108,False,2,0,2011-09-05 17:54:00,Liberty,29,Positive,False,16
1,2,1.109240e+18,FSUHoops,FSU Hoops,2019-03-22 23:59:00,We stand together. Help a Seminole in need.,Text,https://twitter.com/FSUHoops/statuses/11092431...,63235,3911,635,True,1838,49,2009-02-20 15:59:00,Florida State,134,Positive,False,5798
2,3,1.109240e+18,UBmenshoops,UB Men's Basketball,2019-03-22 23:59:00,Bulls Cruise to First-Round Win over Arizona S...,Photo,https://twitter.com/UBmenshoops/statuses/11092...,15485,420,172,True,100,6,2012-08-17 20:32:00,Buffalo,86,Positive,False,526
3,4,1.109240e+18,umichbball,Michigan Men's Basketball,2019-03-22 23:58:00,There is a great challenge ahead of us tomorro...,Photo,https://twitter.com/umichbball/statuses/110924...,309001,224,1687,True,21,1,2009-03-03 16:38:00,Michigan,68,Positive,False,246
4,5,1.109240e+18,ACU_MBB,ACU Mens Basketball,2019-03-22 23:57:00,To our 3 seniors @JayFrank0 @jaren_lewis and @...,Video,https://twitter.com/ACU_MBB/statuses/110924275...,3098,165,42,False,36,0,2013-12-04 16:57:00,Abilene Christi,66,Positive,False,201


In [11]:
basketball_tweets['engagement'] = basketball_tweets['LikeCount'] + basketball_tweets['TWRetweetCount'] + basketball_tweets['TWReplyCount']

In [33]:
basketball_tweets.drop(['Unnamed: 0','ID'],inplace = True, axis = 1)

### Data exploration

In [34]:
basketball_tweets.info()
# no null cells, that's good!

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3316 entries, 0 to 3315
Data columns (total 19 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   AuthorUserName            3316 non-null   object
 1   AuthorDisplayName         3316 non-null   object
 2   Created                   3316 non-null   object
 3   Message                   3316 non-null   object
 4   ContentType               3316 non-null   object
 5   ContextUrl                3316 non-null   object
 6   AuthorFollowerCount       3316 non-null   int64 
 7   LikeCount                 3316 non-null   int64 
 8   TwitterAuthorListedCount  3316 non-null   int64 
 9   TWAuthorIsVerified        3316 non-null   bool  
 10  TWRetweetCount            3316 non-null   int64 
 11  TWReplyCount              3316 non-null   int64 
 12  TWAuthorCreated           3316 non-null   object
 13  Tags                      3316 non-null   object
 14  ZPoints                 

In [35]:
basketball_tweets.describe()

,AuthorFollowerCount,LikeCount,TwitterAuthorListedCount,TWRetweetCount,TWReplyCount,ZPoints,Engagement.Score,engagement
count,3.316000e+03,3316.000000,3316.000000,3316.000000,3316.000000,3316.000000,3316.000000,3316.000000
mean,1.312069e+05,165.606755,447.219542,29.264174,3.103136,44.503920,197.974065,197.974065
std,2.715756e+05,450.235173,448.982939,111.873623,9.443884,28.201735,559.971670,559.971670
min,1.097000e+03,0.000000,32.000000,0.000000,0.000000,6.000000,0.000000,0.000000
25%,8.937000e+03,0.000000,122.000000,0.000000,0.000000,13.000000,0.000000,0.000000
50%,3.936050e+04,33.500000,356.000000,6.000000,1.000000,46.000000,41.000000,41.000000
75%,1.095982e+05,138.000000,593.000000,19.000000,3.000000,64.000000,163.250000,163.250000
max,2.218576e+06,10116.000000,2486.000000,3304.000000,160.000000,144.000000,13580.000000,13580.000000


### 3. Data visualization

In [27]:
hist, edges = np.histogram(basketball_tweets['engagement'], bins = 50)
p = figure(title='Engagement Scores', background_fill_color="#fafafa",plot_width = 800, plot_height = 300)
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], fill_color="navy", line_color="white", alpha=0.5)
show(p)

It looks like engagement scores are very left skewed, it would be good to break out the score into its components: likes, retweets, and replies

In [42]:
hist, edges = np.histogram(basketball_tweets['LikeCount'], bins = 50)
p1 = figure(title='Likes', background_fill_color="#fafafa",plot_width = 800, plot_height = 300)
p1.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], fill_color="navy", line_color="white", alpha=0.5)

hist, edges = np.histogram(basketball_tweets['TWRetweetCount'], bins = 50)
p2 = figure(title='Retweets', background_fill_color="#fafafa",plot_width = 800, plot_height = 300)
p2.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], fill_color="orange", line_color="white", alpha=0.5)

hist, edges = np.histogram(basketball_tweets['TWReplyCount'], bins = 50)
p3 = figure(title='Replies', background_fill_color="#fafafa",plot_width = 800, plot_height = 300)
p3.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], fill_color="pink", line_color="white", alpha=0.5)

show(gridplot([p1,p2,p3], ncols=3, plot_width=265, plot_height=300, toolbar_location=None))

The engagment distribution is similar to the distribution for all three components, with likes having the largest values, followed by retweets and replies.

In [45]:
hist, edges = np.histogram(basketball_tweets['AuthorFollowerCount'], bins = 50)
p = figure(title='Number of Followers', background_fill_color="#fafafa",plot_width = 800, plot_height = 300)
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], fill_color="navy", line_color="white", alpha=0.5)
show(p)